## Data set class

In [43]:
class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: #score of 4 or 5
            return Sentiment.POSITIVE
        
        

## loading the dataset

In [44]:
import json

file_name = './data/sentiment/Books_small.json'

#take of the differece(reviews,review)
reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall'])) #reviewText(text) and overallRating(score)
        
reviews[5].text

'Love the book, great story line, keeps you entertained.for a first novel from this author she did a great job,  Would definitely recommend!'

## Prep data

In [45]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews,test_size=0.33,random_state=42,)

In [48]:
#arrays of text and sentiment for trainning and testing set
train_x = [x.text for x in training]
train_y = [y.sentiment for y in training]

test_x = [x.text for x in test]
test_y = [y.sentiment for y in test]

### Bags of words Vectorization 

In [83]:
from sklearn.feature_extraction.text import CountVectorizer


vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x) ## because we dont want to fit another model so we just transform


print(train_x[0])
print(train_x_vectors[0].toarray())

#we are going to create models for our training set (train_x_vector, train_y)


Vivid characters and descriptions. The author has created a tale that grabs your attention and I couldn't put it down.
[[0 0 0 ... 0 0 0]]


 ## Classification
 
 ### Lear SVM

In [84]:
from sklearn import svm

clf_svm = svm.SVC( kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

test_x[0]


<1x7372 sparse matrix of type '<class 'numpy.int64'>'
	with 36 stored elements in Compressed Sparse Row format>